In [1]:
import numpy as np
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

from tqdm.auto import tqdm

import wandb

import logging
import sys
from torchinfo import summary

c:\conda-envs\envs\DLankle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def setup_logger(name=__name__):
    """
    Sets up a logger that outputs to the console (stdout).
    """
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler(sys.stdout)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

logger = setup_logger()

In [3]:
# Load the dataset
data_folder = "../data"
preped_folder = os.path.join(data_folder, "_preped")

train_data = pd.read_csv(os.path.join(data_folder, 'train_data.csv')).values.tolist()
test_data = pd.read_csv(os.path.join(data_folder, 'test_data.csv')).values.tolist()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to consistent size
    transforms.ToTensor(),           # Convert to tensor [0, 1]
    transforms.Normalize(mean=[0.5], std=[0.5])
])

x_train = []
y_train = []

for img_name, label in train_data:
    img_path = os.path.join(preped_folder, img_name)
    try:
        img = Image.open(img_path).convert('L') # Convert to grayscale
        img_tensor = transform(img)
        x_train.append(img_tensor)
        y_train.append(label)
    except Exception as e:
        logger.info(f"Error loading {img_name}: {e}")

# Stack into tensors
x_train_tensor = torch.stack(x_train)
logger.info(f"Training images shape: {x_train_tensor.shape}")

# Encode labels to integers
label_to_idx = {label: idx for idx, label in enumerate(np.unique(y_train))}
y_train_encoded = [label_to_idx[label] for label in y_train]
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)

logger.info(f"Training labels shape: {y_train_tensor.shape}")
logger.info(f"Label mapping: {label_to_idx}")

2025-12-13 08:58:26,128 - INFO - Training images shape: torch.Size([241, 1, 224, 224])
2025-12-13 08:58:26,130 - INFO - Training labels shape: torch.Size([241])
2025-12-13 08:58:26,130 - INFO - Label mapping: {np.str_('1_Pronacio'): 0, np.str_('2_Neutralis'): 1, np.str_('3_Szupinacio'): 2}


In [4]:
x_test = []
y_test = []

for img_name, label in test_data:
    img_path = os.path.join(preped_folder, img_name)
    try:
        img = Image.open(img_path).convert('L') # Convert to grayscale
        img_tensor = transform(img)
        x_test.append(img_tensor)
        y_test.append(label)
    except Exception as e:
        logger.info(f"Error loading {img_name}: {e}")

x_test_tensor = torch.stack(x_test)
logger.info(f"Test images shape: {x_test_tensor.shape}")
y_test_encoded = [label_to_idx[label] for label in y_test]
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

logger.info(f"Test labels shape: {y_test_tensor.shape}")

2025-12-13 08:58:32,729 - INFO - Test images shape: torch.Size([49, 1, 224, 224])
2025-12-13 08:58:32,730 - INFO - Test labels shape: torch.Size([49])


In [5]:
if torch.cuda.is_available():
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    logger.info(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        logger.info(f"\nGPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        logger.info(f"  Memory: {props.total_memory / 1024**3:.2f} GB")
        logger.info(f"  Compute Capability: {props.major}.{props.minor}")
else:
    logger.info("CUDA not available")

2025-12-13 08:58:32,791 - INFO - CUDA available: True
2025-12-13 08:58:32,792 - INFO - Number of GPUs: 1
2025-12-13 08:58:32,800 - INFO - 
GPU 0: NVIDIA GeForce RTX 4060
2025-12-13 08:58:32,801 - INFO -   Memory: 8.00 GB
2025-12-13 08:58:32,802 - INFO -   Compute Capability: 8.9


In [6]:
batch_size = 16
num_epochs = 70
device = 'cuda' 

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
# wandb login an init
# Login to wandb with API key
load_dotenv()
wandb.login(key=os.getenv("wandbKey"))

def init_wandb():
    # Initialize wandb project
    wandb.init(
        project="ankle-align-inc-model",
        config={
            "batch_size": batch_size,
            "num_epochs": num_epochs,
      
            "architecture": "Custom CNN",
            "dataset": "AnkleAlign",
            "optimizer": "Adam"
        }
    )

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Win 10\_netrc


In [ ]:
net0 = torch.nn.Sequential(
    torch.nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1),   # 224x224 -> 112x112
    torch.nn.ReLU(),
    torch.nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),  # 112x112 -> 56x56
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # 56x56 -> 28x28
    torch.nn.ReLU(),
    torch.nn.AdaptiveAvgPool2d(1),
    torch.nn.Flatten(),
    torch.nn.Linear(32, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 3)                       # Output layer     
).to(device)


def init_weights(m):
    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)

net0.apply(init_weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net0.parameters(), lr=0.01)

summary(net0, input_size=(batch_size, 1, 224, 224))


Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [16, 3]                   --
├─Conv2d: 1-1                            [16, 8, 112, 112]         80
├─ReLU: 1-2                              [16, 8, 112, 112]         --
├─Conv2d: 1-3                            [16, 16, 56, 56]          1,168
├─ReLU: 1-4                              [16, 16, 56, 56]          --
├─Conv2d: 1-5                            [16, 32, 28, 28]          4,640
├─ReLU: 1-6                              [16, 32, 28, 28]          --
├─AdaptiveAvgPool2d: 1-7                 [16, 32, 1, 1]            --
├─Flatten: 1-8                           [16, 32]                  --
├─Linear: 1-9                            [16, 128]                 4,224
├─ReLU: 1-10                             [16, 128]                 --
├─Linear: 1-11                           [16, 64]                  8,256
├─ReLU: 1-12                             [16, 64]                  --
├─L

In [9]:
# Trying to overfit one batch
init_wandb()
one_batch = next(iter(train_loader))
images, labels = one_batch

images = images.to(device)
labels = labels.to(device)

loss_values = []
net0.train()
for epoch in tqdm(range(num_epochs), desc='Training model'):
        pred_logits = net0(images)
        loss = loss_fn(pred_logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_values.append(loss.item())
        wandb.log({
                "epoch": epoch + 1,
                "train_loss": loss.item()
            })
        
wandb.finish()
print(loss_values)

Training model: 100%|██████████| 70/70 [00:00<00:00, 108.06it/s]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇███
train_loss,█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▃▃▄▂▂▁▁
epoch,70
train_loss,0.20578


[0.9189960956573486, 0.9109243750572205, 0.6084246039390564, 0.6110831499099731, 0.5959535837173462, 0.6036111116409302, 0.5776468515396118, 0.5964195132255554, 0.5777697563171387, 0.5779669880867004, 0.5845142006874084, 0.5704132318496704, 0.57173091173172, 0.5740042924880981, 0.562248706817627, 0.5676491260528564, 0.5616258978843689, 0.5558769702911377, 0.5586780309677124, 0.5480696558952332, 0.551421046257019, 0.5429296493530273, 0.5432425737380981, 0.5348362922668457, 0.5359674692153931, 0.524548351764679, 0.530306339263916, 0.5163533091545105, 0.5102657079696655, 0.511725664138794, 0.49767816066741943, 0.48764994740486145, 0.49501532316207886, 0.50182044506073, 0.47447913885116577, 0.45350348949432373, 0.4503435790538788, 0.46593379974365234, 0.5171388387680054, 0.44671016931533813, 0.4155682325363159, 0.4388939142227173, 0.43282079696655273, 0.4105982780456543, 0.3897345960140228, 0.4066815972328186, 0.4314323365688324, 0.3709186315536499, 0.3621132969856262, 0.4061894416809082, 

Net0 looks the most simply CNN which could learn on a 16 image batch, and overfit on this data. Smaller networks were not sifficent enaught to learon on 16 images.

In [10]:
def train_model(network, optimizer, loss_fn, enable_early_stopping=False, patience=5):
    torch.cuda.empty_cache()

    loss_values = []

    if enable_early_stopping:
        early_stopping = EarlyStopping(patience=patience, verbose=True)

    network.train()
    for epoch in tqdm(range(num_epochs), desc='Training model'):
        network.train()
        epoch_loss = 0.0
        num_batches = 0
        for images, target_labels in train_loader:
            images = images.to(device)
            target_labels = target_labels.to(device)

            pred_logits = network(images)
            loss = loss_fn(pred_logits, target_labels)
            epoch_loss += loss.item()
            num_batches += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = epoch_loss / num_batches

        if enable_early_stopping:
            network.eval()
            val_loss = 0.0
            val_batches = 0
            correct = 0
            total = 0
            with torch.no_grad():
                for images, target_labels in val_loader:
                    images = images.to(device)
                    target_labels = target_labels.to(device)
                    
                    pred_logits = network(images)
                    loss = loss_fn(pred_logits, target_labels)
                    val_loss += loss.item()
                    val_batches += 1
                    
                    _, predicted = torch.max(pred_logits, 1)
                    total += target_labels.size(0)
                    correct += (predicted == target_labels).sum().item()
            
            avg_val_loss = val_loss / val_batches
            val_accuracy = correct / total

        # Log metrics
        if enable_early_stopping:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "val_loss": avg_val_loss,
                "val_accuracy": val_accuracy
            })
        else:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss
            })
        loss_values.append(avg_train_loss)
        
        if enable_early_stopping:
            logger.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
        else:
            logger.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")

        # Early stopping check
        if enable_early_stopping:
            early_stopping(avg_val_loss, network)
            if early_stopping.early_stop:
                logger.info("Early stopping triggered")
                network.load_state_dict(early_stopping.best_model)
                break
    
    # Load best model
    if enable_early_stopping and early_stopping.best_model is not None:
        network.load_state_dict(early_stopping.best_model)
        logger.info("Loaded best model weights")

    logger.info(loss_values)

In [11]:
def evaluate_model(network):
    # Training score
    true_labels = y_test_encoded
    predicted_labels = []
    network.eval()
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            outputs = network(images)
            _, predicted = torch.max(outputs, 1)
            predicted_labels.extend(predicted.cpu().numpy())

    accuracy = np.mean([true == pred for true, pred in zip(true_labels, predicted_labels)])
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    logger.info(f"network accuracy: {accuracy * 100:.2f}%")
    logger.info(f"network precision: {precision * 100:.2f}%")
    logger.info(f"network recall: {recall * 100:.2f}%")
    logger.info(f"network F1 score: {f1 * 100:.2f}%")

    logger.info(f"Detailed Classification Report: \n{classification_report(true_labels, predicted_labels)}")

    # Log test metrics
    wandb.log({
        "test_accuracy": accuracy,
        "test_precision": precision,
        "test_recall": recall,
        "test_f1": f1
    })

    wandb.finish()

In [12]:
init_wandb()
train_model(net0, optimizer, loss_fn, enable_early_stopping=False)

Training model:   0%|          | 0/70 [00:00<?, ?it/s]

2025-12-13 08:58:42,088 - INFO - Epoch 1/70, Train Loss: 1.3676


Training model:   1%|▏         | 1/70 [00:00<00:12,  5.48it/s]

2025-12-13 08:58:42,183 - INFO - Epoch 2/70, Train Loss: 1.0051
2025-12-13 08:58:42,270 - INFO - Epoch 3/70, Train Loss: 1.0193


Training model:   4%|▍         | 3/70 [00:00<00:07,  8.75it/s]

...

2025-12-13 08:58:49,461 - INFO - Epoch 69/70, Train Loss: 0.4799


Training model:  99%|█████████▊| 69/70 [00:07<00:00,  8.68it/s]

2025-12-13 08:58:49,565 - INFO - Epoch 70/70, Train Loss: 0.4845


Training model: 100%|██████████| 70/70 [00:07<00:00,  9.14it/s]

2025-12-13 08:58:49,567 - INFO - [1.367609478533268, 1.0050833337008953, 1.019287422299385, 0.9792229644954205, 0.9987912774085999, 1.0225254148244858, 0.9923433512449265, 1.0016814656555653, 0.9874717779457569, 0.9762842319905758, 1.0099237561225891, 0.9834324643015862, 0.9767465516924858, 1.0197370164096355, 0.9611853286623955, 0.9505730979144573, 0.9390123076736927, 1.0395533852279186, 1.0613774731755257, 0.9847182855010033, 0.949910644441843, 1.0370331592857838, 0.9700660593807697, 0.9628367051482201, 0.9305558316409588, 0.9515724293887615, 0.9399220794439316, 0.9337956011295319, 0.9297220185399055, 0.9591145627200603, 0.8830360732972622, 0.9720307663083076, 0.9694576673209667, 0.9415075182914734, 0.8797676488757133, 0.8724886253476143, 0.8441466502845287, 0.8001011405140162, 0.8206600099802017, 0.7307148575782776, 1.3516437225043774, 1.1947062239050865, 0.9481813460588455, 0.9477338530123234, 0.8607966676354408, 0.8279795423150063, 0.8270625714212656, 0.7394652627408504, 0.7450531

Net0 can learn on one batch of 16 images, but can not learn on all the provided train data, to simple for 200+ images.

In [ ]:
net1 = torch.nn.Sequential(
    torch.nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1),   # 224x224 -> 112x112   // (3x3x1)x8
    torch.nn.ReLU(),
    torch.nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),  # 112x112 -> 56x56     // (3x3x8)x16
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # 56x56 -> 28x28      // (3x3x16)x32
    torch.nn.ReLU(),
    torch.nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),  # 28x28 -> 14x14      // (3x3x32)x32
    torch.nn.ReLU(),
    torch.nn.AdaptiveAvgPool2d(1),
    torch.nn.Flatten(),
    torch.nn.Linear(32, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 3)                       # Output layer     
).to(device)

net1.apply(init_weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net1.parameters(), lr=0.001)

summary(net1, input_size=(batch_size, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [16, 3]                   --
├─Conv2d: 1-1                            [16, 8, 112, 112]         80
├─ReLU: 1-2                              [16, 8, 112, 112]         --
├─Conv2d: 1-3                            [16, 16, 56, 56]          1,168
├─ReLU: 1-4                              [16, 16, 56, 56]          --
├─Conv2d: 1-5                            [16, 32, 28, 28]          4,640
├─ReLU: 1-6                              [16, 32, 28, 28]          --
├─Conv2d: 1-7                            [16, 32, 14, 14]          9,248
├─ReLU: 1-8                              [16, 32, 14, 14]          --
├─AdaptiveAvgPool2d: 1-9                 [16, 32, 1, 1]            --
├─Flatten: 1-10                          [16, 32]                  --
├─Linear: 1-11                           [16, 128]                 4,224
├─ReLU: 1-12                             [16, 128]                 --
├─L

In [14]:
init_wandb()
train_model(net1, optimizer, loss_fn, enable_early_stopping=False)
wandb.finish()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
train_loss,█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▅▅▅▄▄▄▃█▅▅▄▃▃▅▄▃▃▃▂▃▂▃▁
epoch,70
train_loss,0.48453


Training model:   0%|          | 0/70 [00:00<?, ?it/s]

2025-12-13 08:58:53,286 - INFO - Epoch 1/70, Train Loss: 1.0399


Training model:   1%|▏         | 1/70 [00:00<00:12,  5.31it/s]

...

Training model:  99%|█████████▊| 69/70 [00:08<00:00,  7.28it/s]

2025-12-13 08:59:01,971 - INFO - Epoch 70/70, Train Loss: 0.4017


Training model: 100%|██████████| 70/70 [00:08<00:00,  7.89it/s]

2025-12-13 08:59:01,973 - INFO - [1.0399250909686089, 1.0587920546531677, 1.0044136866927147, 0.9897469095885754, 0.9755896776914597, 0.991124402731657, 0.9646955281496048, 1.04264210537076, 0.9857311733067036, 0.9359408151358366, 0.9648342840373516, 0.9729919619858265, 0.9673570320010185, 0.9942279644310474, 0.9509303942322731, 0.9442183263599873, 0.9756307825446129, 0.9840870201587677, 0.9559366554021835, 0.9507177956402302, 0.948088962584734, 0.9401596039533615, 0.9618076607584953, 0.9211361669003963, 0.927639152854681, 0.9469483271241188, 0.9128565788269043, 0.8862023800611496, 0.9155723638832569, 0.9106504619121552, 0.8601327519863844, 0.8929817080497742, 0.9369538389146328, 0.876258909702301, 0.8895190432667732, 0.8098578378558159, 0.8024081028997898, 0.8345138542354107, 0.8259834088385105, 0.8053876645863056, 0.7939435169100761, 0.8447823598980904, 0.7518744012340903, 0.7166483849287033, 0.7607695125043392, 0.7263949736952782, 0.6663938499987125, 0.6699719466269016, 0.6237642664

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
train_loss,██▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▅▅▆▅▅▅▄▄▄▃▃▃▂▃▂▁▁▂
epoch,70
train_loss,0.40171


Net1 could learn on all the provided train data, but it took the model over 50 epoch to converge below 0.5 loss on train set. So I think I will create 1 more little bit more complex network.

In [ ]:
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1),   # 224x224 -> 112x112   // (3x3x1)x8
    torch.nn.ReLU(),
    torch.nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),  # 112x112 -> 56x56     // (3x3x8)x16
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # 56x56 -> 28x28      // (3x3x16)x32
    torch.nn.ReLU(),
    torch.nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # 28x28 -> 14x14      // (3x3x32)x64
    torch.nn.ReLU(),
    torch.nn.Conv2d(64, 32, kernel_size=3, stride=2, padding=1),  # 14x14 -> 7x7        // (3x3x64)x32
    torch.nn.ReLU(),
    torch.nn.AdaptiveAvgPool2d(1),
    torch.nn.Flatten(),
    torch.nn.Linear(32, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 3)                       # Output layer     
).to(device)

net2.apply(init_weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net2.parameters(), lr=0.001)

summary(net2, input_size=(batch_size, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [16, 3]                   --
├─Conv2d: 1-1                            [16, 8, 112, 112]         80
├─ReLU: 1-2                              [16, 8, 112, 112]         --
├─Conv2d: 1-3                            [16, 16, 56, 56]          1,168
├─ReLU: 1-4                              [16, 16, 56, 56]          --
├─Conv2d: 1-5                            [16, 32, 28, 28]          4,640
├─ReLU: 1-6                              [16, 32, 28, 28]          --
├─Conv2d: 1-7                            [16, 64, 14, 14]          18,496
├─ReLU: 1-8                              [16, 64, 14, 14]          --
├─Conv2d: 1-9                            [16, 32, 7, 7]            18,464
├─ReLU: 1-10                             [16, 32, 7, 7]            --
├─AdaptiveAvgPool2d: 1-11                [16, 32, 1, 1]            --
├─Flatten: 1-12                          [16, 32]                  --
├

In [16]:
init_wandb()
train_model(net2, optimizer, loss_fn, enable_early_stopping=False)
evaluate_model(net2)

Training model:   0%|          | 0/70 [00:00<?, ?it/s]

2025-12-13 08:59:04,957 - INFO - Epoch 1/70, Train Loss: 0.9957


Training model:   1%|▏         | 1/70 [00:00<00:17,  4.04it/s]

2025-12-13 08:59:05,129 - INFO - Epoch 2/70, Train Loss: 1.0129


Training model:   3%|▎         | 2/70 [00:00<00:13,  4.92it/s]

...

2025-12-13 08:59:13,960 - INFO - [0.9956907853484154, 1.0128592774271965, 1.0744765773415565, 1.0224140472710133, 0.9865722917020321, 0.9618925675749779, 0.9555903635919094, 0.9642202518880367, 0.9374573156237602, 0.9424333684146404, 0.9132138900458813, 0.9232466984540224, 0.9042926207184792, 0.897444412112236, 0.8278295639902353, 0.8496574871242046, 0.8351486176252365, 0.7468331698328257, 0.6240808647125959, 0.5798384752124548, 0.6294200960546732, 0.6197226643562317, 0.5584932754281908, 0.4672447012271732, 0.3648925491143018, 0.32740815496072173, 0.29216101253405213, 0.2215290897875093, 0.24248065007850528, 0.3481039176695049, 0.31198797561228275, 0.26836151140742004, 0.1757689490239045, 0.13624115788843483, 0.11561787932259904, 0.09073313918088388, 0.10790391359478235, 0.1017022569428434, 0.09758510446408764, 0.11143907031510025, 0.09241706322063692, 0.0689141603392045, 0.06891027855454013, 0.07312596451083664, 0.06802698915998917, 0.06105668313102797, 0.0730885010707425, 0.087200891


c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


2025-12-13 08:59:14,010 - INFO - network F1 score: 34.71%
2025-12-13 08:59:14,023 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.22      0.86      0.35         7
           1       0.90      0.21      0.35        42
           2       0.00      0.00      0.00         0

    accuracy                           0.31        49
   macro avg       0.37      0.36      0.23        49
weighted avg       0.80      0.31      0.35        49



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
wandb: ERROR The nbformat package was not foun

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇███
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,███▇▇▇▇▇▇▆▆▅▅▅▅▃▃▃▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,70
test_accuracy,0.30612
test_f1,0.34712
test_precision,0.80317
test_recall,0.30612


Net2 seems the best in convergence but it definitely overfits, so the next step is to solve this with net3.

In [17]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0001, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model = None
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                logger.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
            self.counter = 0


In [18]:
batch_size = 32
net3 = torch.nn.Sequential(
    torch.nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),      # (3x3x1)x8
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 224x224 -> 112x112

    torch.nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),       # (3x3x8)x16
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 112x112 -> 56x56

    torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),       # (3x3x16)x32
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 56x56 -> 28x28 

    torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),       # (3x3x32)x64
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 28x28 -> 14x14 

    torch.nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),       # (3x3x64)x32
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 14x14 -> 7x7

    torch.nn.AdaptiveAvgPool2d(1),
    torch.nn.Flatten(),
    torch.nn.Linear(32, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 3)                       # Output layer     
).to(device)

net3.apply(init_weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net3.parameters(), lr=0.001)

summary(net3, input_size=(batch_size, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 3]                   --
├─Conv2d: 1-1                            [32, 8, 224, 224]         80
├─ReLU: 1-2                              [32, 8, 224, 224]         --
├─MaxPool2d: 1-3                         [32, 8, 112, 112]         --
├─Conv2d: 1-4                            [32, 16, 112, 112]        1,168
├─ReLU: 1-5                              [32, 16, 112, 112]        --
├─MaxPool2d: 1-6                         [32, 16, 56, 56]          --
├─Conv2d: 1-7                            [32, 32, 56, 56]          4,640
├─ReLU: 1-8                              [32, 32, 56, 56]          --
├─MaxPool2d: 1-9                         [32, 32, 28, 28]          --
├─Conv2d: 1-10                           [32, 64, 28, 28]          18,496
├─ReLU: 1-11                             [32, 64, 28, 28]          --
├─MaxPool2d: 1-12                        [32, 64, 14, 14]          --
├─Con

In [19]:
init_wandb()
train_model(net3, optimizer, loss_fn, enable_early_stopping=False)
evaluate_model(net3)

Training model:   0%|          | 0/70 [00:00<?, ?it/s]

2025-12-13 08:59:17,196 - INFO - Epoch 1/70, Train Loss: 1.1933


Training model:   1%|▏         | 1/70 [00:00<00:29,  2.33it/s]

2025-12-13 08:59:17,335 - INFO - Epoch 2/70, Train Loss: 1.0336


Training model:   3%|▎         | 2/70 [00:00<00:17,  3.87it/s]

...

2025-12-13 08:59:27,908 - INFO - [1.1932735219597816, 1.0335881970822811, 0.9807348139584064, 1.042787965387106, 0.976081408560276, 0.9911840967833996, 0.971342895179987, 0.9815533012151718, 1.005672175437212, 0.9501973241567612, 0.9659802876412868, 1.0056226253509521, 0.9348855875432491, 0.9265397377312183, 0.9070290513336658, 0.9525636471807957, 0.870769340544939, 0.8230959130451083, 0.8252803646028042, 0.8178792037069798, 0.7936655506491661, 0.8643611613661051, 0.8070287359878421, 0.8448962569236755, 0.7694230210036039, 0.706070626154542, 0.7387008685618639, 0.8054158389568329, 0.7323718369007111, 0.9050506241619587, 0.8509819768369198, 0.7616352587938309, 0.7051434237509966, 0.6489369096234441, 0.5854946076869965, 0.5620360033353791, 0.48575551621615887, 0.6040581102715805, 0.5158599717542529, 0.5746682249009609, 0.7559193912893534, 0.5731692090630531, 0.4553545406088233, 0.43126018065959215, 0.459331464022398, 0.36239993944764093, 0.4620612794533372, 0.5370859894901514, 0.37309488


c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


2025-12-13 08:59:27,944 - INFO - network precision: 89.71%
2025-12-13 08:59:27,945 - INFO - network recall: 51.02%
2025-12-13 08:59:27,946 - INFO - network F1 score: 57.68%
2025-12-13 08:59:27,962 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.28      1.00      0.44         7
           1       1.00      0.43      0.60        42
           2       0.00      0.00      0.00         0

    accuracy                           0.51        49
   macro avg       0.43      0.48      0.35        49
weighted avg       0.90      0.51      0.58        49



c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
wandb: ERROR The nbformat package was not foun

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,█▇▇▇▇▇▆▇▆▆▆▆▆▆▅▆▅▅▅▅▄▄▄▅▄▃▃▃▄▃▂▁▁▁▁▁▁▁▁▁
epoch,70
test_accuracy,0.5102
test_f1,0.57679
test_precision,0.89714
test_recall,0.5102


Net3 intorduced maxpool layers after each conv layer and acheved better generalisation, more over batch size could be inreased to 32, and still perfomed godd on it unlike previous networks.

In [20]:
# Use train test split, for getting validation metrics during training
x_train_tensor, x_val_tensor, y_train_tensor, y_val_tensor = train_test_split(
    x_train_tensor, y_train_tensor, test_size=0.2, random_state=42, stratify=y_train_tensor)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [29]:
def init_weights(m):
    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif isinstance(m, torch.nn.BatchNorm2d):
        torch.nn.init.constant_(m.weight, 1)
        torch.nn.init.constant_(m.bias, 0)

In [ ]:
net4 = torch.nn.Sequential(
    torch.nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),      # (3x3x1)x8
    torch.nn.BatchNorm2d(8),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 224x224 -> 112x112

    torch.nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),       # (3x3x8)x16
    torch.nn.BatchNorm2d(16),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 112x112 -> 56x56

    torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),       # (3x3x16)x32
    torch.nn.BatchNorm2d(32),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 56x56 -> 28x28 

    torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),       # (3x3x32)x64
    torch.nn.BatchNorm2d(64),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 28x28 -> 14x14 

    torch.nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),       # (3x3x64)x32
    torch.nn.BatchNorm2d(32),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 14x14 -> 7x7

    torch.nn.AdaptiveAvgPool2d(1),
    torch.nn.Flatten(),
    torch.nn.Linear(32, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 3)                       # Output layer     
).to(device)

net4.apply(init_weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net4.parameters(), lr=0.0005)

summary(net4, input_size=(batch_size, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 3]                   --
├─Conv2d: 1-1                            [32, 8, 224, 224]         80
├─BatchNorm2d: 1-2                       [32, 8, 224, 224]         16
├─ReLU: 1-3                              [32, 8, 224, 224]         --
├─MaxPool2d: 1-4                         [32, 8, 112, 112]         --
├─Conv2d: 1-5                            [32, 16, 112, 112]        1,168
├─BatchNorm2d: 1-6                       [32, 16, 112, 112]        32
├─ReLU: 1-7                              [32, 16, 112, 112]        --
├─MaxPool2d: 1-8                         [32, 16, 56, 56]          --
├─Conv2d: 1-9                            [32, 32, 56, 56]          4,640
├─BatchNorm2d: 1-10                      [32, 32, 56, 56]          64
├─ReLU: 1-11                             [32, 32, 56, 56]          --
├─MaxPool2d: 1-12                        [32, 32, 28, 28]          --
├─Conv2d:

In [37]:
init_wandb()
train_model(net4, optimizer, loss_fn, enable_early_stopping=True, patience=5)
evaluate_model(net4)

Training model:   0%|          | 0/70 [00:00<?, ?it/s]

2025-12-13 09:15:41,900 - INFO - Epoch 1/70, Train Loss: 1.0242, Val Loss: 1.0488, Val Acc: 0.4082


Training model:   1%|▏         | 1/70 [00:00<00:29,  2.36it/s]

2025-12-13 09:15:42,036 - INFO - Epoch 2/70, Train Loss: 0.9303, Val Loss: 1.0724, Val Acc: 0.4694
2025-12-13 09:15:42,036 - INFO - EarlyStopping counter: 1 out of 5


Training model:   3%|▎         | 2/70 [00:00<00:17,  3.93it/s]

...

2025-12-13 09:15:43,943 - INFO - Epoch 16/70, Train Loss: 0.4987, Val Loss: 1.0972, Val Acc: 0.6531
2025-12-13 09:15:43,944 - INFO - EarlyStopping counter: 5 out of 5
2025-12-13 09:15:43,944 - INFO - Early stopping triggered


Training model:  21%|██▏       | 15/70 [00:02<00:09,  6.06it/s]

2025-12-13 09:15:43,957 - INFO - Loaded best model weights
2025-12-13 09:15:43,958 - INFO - [1.0241894920667012, 0.9302704135576884, 0.8767741024494171, 0.8448319633801779, 0.8033467233181, 0.7947917580604553, 0.7572037974993387, 0.7275828321774801, 0.7008424401283264, 0.6737854778766632, 0.6508257488409678, 0.6046920021375021, 0.5815143783887228, 0.541930745045344, 0.5239527573188146, 0.49871138234933216]
2025-12-13 09:15:44,010 - INFO - network accuracy: 46.94%
2025-12-13 09:15:44,011 - INFO - network precision: 80.39%
2025-12-13 09:15:44,011 - INFO - network recall: 46.94%
2025-12-13 09:15:44,012 - INFO - network F1 score: 54.70%
2025-12-13 09:15:44,029 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.23      0.71      0.34         7
           1       0.90      0.43      0.58        42
           2       0.00      0.00      0.00         0

    accuracy                           0.47        49
   macro avg       0.


c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\conda-envs\envs\DLankle\Lib\site-packages\

epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,█▇▆▆▅▅▄▄▄▃▃▂▂▂▁▁
val_accuracy,▁▂▁▁▃▄▆▆▄▆▆▃█▅▆▇
val_loss,▄▆▆▄▃▃▃▃▃▅▁▂▃▅▃█
epoch,16
test_accuracy,0.46939
test_f1,0.54696


Net4 introduced some batch normalization to help with the overfitting problem. The learning rate also got decreased.

In [ ]:
net5 = torch.nn.Sequential(
    torch.nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),      # (3x3x1)x8
    torch.nn.BatchNorm2d(8),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 224x224 -> 112x112

    torch.nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),       # (3x3x8)x16
    torch.nn.BatchNorm2d(16),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 112x112 -> 56x56

    torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),       # (3x3x16)x32
    torch.nn.BatchNorm2d(32),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 56x56 -> 28x28 

    torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),       # (3x3x32)x64
    torch.nn.BatchNorm2d(64),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 28x28 -> 14x14 

    torch.nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),       # (3x3x64)x32
    torch.nn.BatchNorm2d(32),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),    # 14x14 -> 7x7

    torch.nn.AdaptiveAvgPool2d(1),
    torch.nn.Flatten(),
    torch.nn.Linear(32, 128),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(64, 3)                       # Output layer     
).to(device)

net5.apply(init_weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net5.parameters(), lr=0.0005)

summary(net5, input_size=(batch_size, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 3]                   --
├─Conv2d: 1-1                            [32, 8, 224, 224]         80
├─BatchNorm2d: 1-2                       [32, 8, 224, 224]         16
├─ReLU: 1-3                              [32, 8, 224, 224]         --
├─MaxPool2d: 1-4                         [32, 8, 112, 112]         --
├─Conv2d: 1-5                            [32, 16, 112, 112]        1,168
├─BatchNorm2d: 1-6                       [32, 16, 112, 112]        32
├─ReLU: 1-7                              [32, 16, 112, 112]        --
├─MaxPool2d: 1-8                         [32, 16, 56, 56]          --
├─Conv2d: 1-9                            [32, 32, 56, 56]          4,640
├─BatchNorm2d: 1-10                      [32, 32, 56, 56]          64
├─ReLU: 1-11                             [32, 32, 56, 56]          --
├─MaxPool2d: 1-12                        [32, 32, 28, 28]          --
├─Conv2d:

In [45]:
init_wandb()
train_model(net5, optimizer, loss_fn, enable_early_stopping=True, patience=5)
evaluate_model(net5)

Training model:   0%|          | 0/70 [00:00<?, ?it/s]

2025-12-13 09:48:41,084 - INFO - Epoch 1/70, Train Loss: 1.4772, Val Loss: 1.0236, Val Acc: 0.4694


Training model:   1%|▏         | 1/70 [00:00<00:28,  2.41it/s]

2025-12-13 09:48:41,225 - INFO - Epoch 2/70, Train Loss: 1.3053, Val Loss: 1.0214, Val Acc: 0.4694


Training model:   3%|▎         | 2/70 [00:00<00:17,  3.93it/s]

...

2025-12-13 09:48:42,100 - INFO - Epoch 9/70, Train Loss: 1.1417, Val Loss: 1.0764, Val Acc: 0.5714
2025-12-13 09:48:42,100 - INFO - EarlyStopping counter: 5 out of 5
2025-12-13 09:48:42,100 - INFO - Early stopping triggered


Training model:  11%|█▏        | 8/70 [00:01<00:11,  5.59it/s]

2025-12-13 09:48:42,104 - INFO - Loaded best model weights
2025-12-13 09:48:42,105 - INFO - [1.4771883487701416, 1.3052693406740825, 1.2189250588417053, 1.1031062801678975, 1.1549132664998372, 1.1303402582804363, 1.1344122886657715, 1.1339189608891804, 1.1417464415232341]
2025-12-13 09:48:42,130 - INFO - network accuracy: 67.35%
2025-12-13 09:48:42,131 - INFO - network precision: 83.76%
2025-12-13 09:48:42,132 - INFO - network recall: 67.35%
2025-12-13 09:48:42,132 - INFO - network F1 score: 72.16%
2025-12-13 09:48:42,141 - INFO - Detailed Classification Report: 
              precision    recall  f1-score   support

           0       0.26      0.71      0.38         7
           1       0.93      0.67      0.78        42

    accuracy                           0.67        49
   macro avg       0.60      0.69      0.58        49
weighted avg       0.84      0.67      0.72        49




wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▂▃▄▅▅▆▇█
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,█▅▃▁▂▂▂▂▂
val_accuracy,▃▃▃▁▁▆▇▇█
val_loss,▂▂▁▁▂▃▆██
epoch,9
test_accuracy,0.67347
test_f1,0.72161


Net5 introdiced dropout to the fully connected layers but it did not imrpoved the model's performance compared to the previous net4.